In [ ]:
import os
import sys
os.chdir('/local/home/dhaziza/entrack')
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="4"
sys.path.append('/local/home/dhaziza/entrack/')

In [ ]:
from src.preprocessing import preprocess_all
preprocess_all()

In [ ]:
import tensorflow as tf
from src.model import model_fn
from src.input import train_input, test_input
config = tf.estimator.RunConfig(model_dir="data/model")
nn = tf.estimator.Estimator(model_fn=model_fn, params={}, config=config)
tf.logging.set_verbosity(tf.logging.INFO)
for i in range(5):
    nn.train(input_fn=train_input)
    nn.evaluate(input_fn=test_input)


In [ ]:
it = train_input()
sess = tf.Session()
dat = sess.run(it['mri'])
import matplotlib.pyplot as plt
%matplotlib inline
for i in range(8):
    plt.imshow(dat[i, :,:,50])
    plt.show()